<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled296.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# ============================================================
# Paper X (AOI-gap) — FULL reproducible run
# - includes strict ZERO controls
# - saves ONE summary NPZ + figure PNGs
# - avoids np.trapz (uses np.trapezoid)
# ============================================================

# -------------------------
# Utils
# -------------------------
def sigmoid(kappa, x):
    z = np.clip(-kappa * x, -60.0, 60.0)
    return 1.0 / (1.0 + np.exp(z))

def run_channel(V, kappa, Vc):
    return sigmoid(kappa, V - Vc)

def aoi_positive_area(x):
    """AOI proxy: trapezoid integral of positive excursion above mean."""
    y = np.maximum(x - np.mean(x), 0.0)
    return float(np.trapezoid(y))

def ensure_dir(path):
    d = os.path.dirname(path)
    if d:
        os.makedirs(d, exist_ok=True)

# -------------------------
# Core AB vs BA schedule
# -------------------------
def run_AB_BA(
    V,
    Tsteps=120,
    charge_params=(2.4, 0.0),
    heat_params=(1.0, -0.15),
    p_thresh=0.7,
    alpha=0.0,
    asym_threshold=False,
    delta_thr=0.10,
    omega=0.8,
    drive_amp=0.25,
    seed=0,
):
    """
    AB vs BA:
      - AB: charge reads first, writes back, then heat reads
      - BA: heat reads first, writes back, then charge reads

    alpha=0 -> unordered control (no write-back)
    asym_threshold -> shifts VcC and VcH by +/- delta_thr
    """
    rng = np.random.default_rng(seed)  # present for future extension; not used by default
    V = np.asarray(V, dtype=float).copy()

    kC, VcC = charge_params
    kH, VcH = heat_params

    if asym_threshold:
        VcC = VcC + float(delta_thr)
        VcH = VcH - float(delta_thr)

    def drive(t):
        return (1.0 + float(drive_amp) * np.cos(float(omega) * t))

    def schedule(order):
        rC = np.zeros(Tsteps, dtype=float)
        rH = np.zeros(Tsteps, dtype=float)
        dR = np.zeros(Tsteps, dtype=float)  # reach difference

        for t in range(Tsteps):
            Vt = drive(t) * V

            if order == "AB":
                pC = run_channel(Vt, kC, VcC)
                Vt2 = Vt - float(alpha) * (pC - pC.mean()) if alpha != 0.0 else Vt
                pH = run_channel(Vt2, kH, VcH)
            else:  # "BA"
                pH = run_channel(Vt, kH, VcH)
                Vt2 = Vt - float(alpha) * (pH - pH.mean()) if alpha != 0.0 else Vt
                pC = run_channel(Vt2, kC, VcC)

            rC[t] = float(np.mean(pC > p_thresh))
            rH[t] = float(np.mean(pH > p_thresh))
            dR[t] = rC[t] - rH[t]

        return rC, rH, dR

    rC_AB, rH_AB, dR_AB = schedule("AB")
    rC_BA, rH_BA, dR_BA = schedule("BA")

    AOI_AB = aoi_positive_area(dR_AB)
    AOI_BA = aoi_positive_area(dR_BA)

    out = dict(
        AOI_gap_L1=float(abs(AOI_AB - AOI_BA)),
        AOI_gap_signed=float(AOI_AB - AOI_BA),
        AOI_gap_final=float(dR_AB[-1] - dR_BA[-1]),
        AOI_AB=float(AOI_AB),
        AOI_BA=float(AOI_BA),
        rC_AB=rC_AB, rH_AB=rH_AB, dR_AB=dR_AB,
        rC_BA=rC_BA, rH_BA=rH_BA, dR_BA=dR_BA,
        alpha=float(alpha),
        p_thresh=float(p_thresh),
        asym_threshold=bool(asym_threshold),
        delta_thr=float(delta_thr),
        omega=float(omega),
        drive_amp=float(drive_amp),
        seed=int(seed),
    )
    return out

# -------------------------
# Figure helpers
# -------------------------
def plot_quadrant_bars(stage_null, stage_ordered, labels, short_labels, out_png, p_thresh, alpha_null, alpha_ord):
    """
    Two-panel compact figure:
      Left: null controls (alpha=0) — show bars even if all zero
      Right: ordered (alpha>0)
    """
    ensure_dir(out_png)

    null_vals = np.array([stage_null[k]["AOI_gap_L1"] for k in labels], dtype=float)
    ord_vals  = np.array([stage_ordered[k]["AOI_gap_L1"] for k in labels], dtype=float)

    x = np.arange(len(labels), dtype=float)
    w = 0.72

    fig, axes = plt.subplots(1, 2, figsize=(7.0, 2.6), dpi=240, constrained_layout=True)

    # ---- Left (null)
    ax = axes[0]
    ax.bar(x, null_vals, width=w, facecolor="none", edgecolor="black", linewidth=1.6)
    ax.axhline(0.0, linewidth=1.0)
    ax.set_title(f"Controls (alpha={alpha_null:g})")
    ax.set_ylabel("AOI-gap")
    ax.set_xticks(x)
    ax.set_xticklabels(short_labels)

    # force visible scale even if exactly zero
    vmax = float(np.max(np.abs(null_vals)))
    span = max(0.02, 5.0 * vmax)
    ax.set_ylim(-span, span)

    for i, v in enumerate(null_vals):
        ax.text(i, v + 0.06*span, f"{v:.3f}", ha="center", va="bottom", fontsize=9)

    ax.text(0.02, 0.92, "Strict nulls shown\n(not omitted)", transform=ax.transAxes,
            ha="left", va="top", fontsize=9)

    # ---- Right (ordered)
    ax = axes[1]
    ax.bar(x, ord_vals, width=w, linewidth=0.0)
    ax.axhline(0.0, linewidth=1.0)
    ax.set_title(f"Ordered (alpha={alpha_ord:g})")
    ax.set_ylabel("AOI-gap")
    ax.set_xticks(x)
    ax.set_xticklabels(short_labels)

    top = max(1e-6, 1.18 * float(np.max(ord_vals)))
    ax.set_ylim(0.0, top)

    for i, v in enumerate(ord_vals):
        ax.text(i, v + 0.03*top, f"{v:.3f}", ha="center", va="bottom", fontsize=9)

    fig.suptitle(f"AOI-gap quadrant (p_thresh={p_thresh:g})", y=1.04, fontsize=12)
    plt.savefig(out_png, bbox_inches="tight")
    plt.close(fig)

def plot_traces_stacked(stage_ordered, labels, out_png, title):
    """
    Stacked traces: for each case, plot dR_AB (solid) and dR_BA (dashed).
    (This is the cleanest witness of non-commutativity for readers.)
    """
    ensure_dir(out_png)

    n = len(labels)
    fig, axes = plt.subplots(n, 1, figsize=(7.0, 6.6), dpi=240, sharex=True, constrained_layout=True)
    if n == 1:
        axes = [axes]

    for ax, lab in zip(axes, labels):
        dR_AB = np.asarray(stage_ordered[lab]["dR_AB"], dtype=float)
        dR_BA = np.asarray(stage_ordered[lab]["dR_BA"], dtype=float)

        ax.plot(dR_AB, label="AB", linewidth=1.4)
        ax.plot(dR_BA, "--", label="BA", linewidth=1.4)
        ax.axhline(0.0, linewidth=0.9)
        ax.set_ylabel(lab.replace("_", "\n"), rotation=0, labelpad=48, va="center", fontsize=8)

    axes[-1].set_xlabel("t")
    axes[0].legend(loc="upper right", fontsize=9, frameon=False)
    fig.suptitle(title, y=1.01, fontsize=12)

    plt.savefig(out_png, bbox_inches="tight")
    plt.close(fig)

# -------------------------
# FULL run: includes zero controls
# -------------------------
def run_paperX_full(
    outdir="outputs",
    Nx=23, Ny=23,
    Tsteps=120,
    p_thresh=0.7,
    alpha_null=0.0,
    alpha_ord=0.8,
    chi=0.6,
    delta_thr=0.10,
    seed=0,
):
    ensure_dir(os.path.join(outdir, "dummy.txt"))

    n = int(Nx) * int(Ny)
    x = np.linspace(0.0, 2.0*np.pi, n, endpoint=False)

    # Base minimal field
    V_base = np.cos(x)

    # Minimal "chirality" perturbation (handed harmonic)
    V_chiral = V_base + float(chi) * np.sin(2.0 * x)

    labels = [
        "SYM_graphene_graphene",
        "ASYM_threshold_only",
        "ASYM_chirality_only",
        "ASYM_chi_plus_threshold",
    ]
    short_labels = ["SYM", "THR", "CHI", "CHI+THR"]

    cases = [
        (V_base,   False),
        (V_base,   True),
        (V_chiral, False),
        (V_chiral, True),
    ]

    # -------- Stage NULL: strict controls
    stage_null = {}
    for lab, (V, asym_thr) in zip(labels, cases):
        stage_null[lab] = run_AB_BA(
            V=V,
            Tsteps=Tsteps,
            p_thresh=p_thresh,
            alpha=alpha_null,
            asym_threshold=asym_thr,
            delta_thr=delta_thr,
            seed=seed,
        )

    # -------- Stage ORDERED: non-commutative ordering on
    stage_ord = {}
    for lab, (V, asym_thr) in zip(labels, cases):
        stage_ord[lab] = run_AB_BA(
            V=V,
            Tsteps=Tsteps,
            p_thresh=p_thresh,
            alpha=alpha_ord,
            asym_threshold=asym_thr,
            delta_thr=delta_thr,
            seed=seed,
        )

    # -------- Save one NPZ summary
    out_npz = os.path.join(outdir, f"aoi_gap_quadrant_summary_alpha{alpha_null:g}_vs_alpha{alpha_ord:g}_pth{p_thresh:.2f}.npz")
    np.savez_compressed(
        out_npz,
        stage_null=stage_null,
        stage_ordered=stage_ord,
        labels=np.array(labels, dtype=object),
        short_labels=np.array(short_labels, dtype=object),
        Nx=int(Nx), Ny=int(Ny),
        Tsteps=int(Tsteps),
        p_thresh=float(p_thresh),
        alpha_null=float(alpha_null),
        alpha_ord=float(alpha_ord),
        chi=float(chi),
        delta_thr=float(delta_thr),
        seed=int(seed),
    )

    # -------- Figures
    fig2_png = os.path.join(outdir, "fig2_AOI_gap_quadrant_controls_vs_ordered.png")
    plot_quadrant_bars(
        stage_null=stage_null,
        stage_ordered=stage_ord,
        labels=labels,
        short_labels=short_labels,
        out_png=fig2_png,
        p_thresh=p_thresh,
        alpha_null=alpha_null,
        alpha_ord=alpha_ord,
    )

    fig3_png = os.path.join(outdir, "fig3_dReach_traces_ordered_stacked.png")
    plot_traces_stacked(
        stage_ordered=stage_ord,
        labels=labels,
        out_png=fig3_png,
        title=f"Ordered-access traces (alpha={alpha_ord:g}, p_thresh={p_thresh:g})",
    )

    # -------- Console summary (so you can paste into paper SI)
    print("=== AOI-gap quadrant (strict controls + ordered) ===")
    for lab in labels:
        z = stage_null[lab]["AOI_gap_L1"]
        o = stage_ord[lab]["AOI_gap_L1"]
        print(f"{lab:28s}  control={z: .6f}   ordered={o: .6f}")

    print("\nSaved:", out_npz)
    print("Saved:", fig2_png)
    print("Saved:", fig3_png)

    return out_npz, fig2_png, fig3_png

# ============================================================
# RUN (default witness parameters)
# ============================================================
run_paperX_full()

=== AOI-gap quadrant (strict controls + ordered) ===
SYM_graphene_graphene         control= 0.000000   ordered= 1.503261
ASYM_threshold_only           control= 0.000000   ordered= 3.676244
ASYM_chirality_only           control= 0.000000   ordered= 1.780120
ASYM_chi_plus_threshold       control= 0.000000   ordered= 1.292155

Saved: outputs/aoi_gap_quadrant_summary_alpha0_vs_alpha0.8_pth0.70.npz
Saved: outputs/fig2_AOI_gap_quadrant_controls_vs_ordered.png
Saved: outputs/fig3_dReach_traces_ordered_stacked.png


('outputs/aoi_gap_quadrant_summary_alpha0_vs_alpha0.8_pth0.70.npz',
 'outputs/fig2_AOI_gap_quadrant_controls_vs_ordered.png',
 'outputs/fig3_dReach_traces_ordered_stacked.png')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# ============================================================
# Paper X (AOI-gap) — FULL reproducible run
# - includes strict ZERO controls
# - saves ONE summary NPZ + figure PNGs
# - avoids np.trapz (uses np.trapezoid)
# ============================================================

# -------------------------
# Utils
# -------------------------
def sigmoid(kappa, x):
    z = np.clip(-kappa * x, -60.0, 60.0)
    return 1.0 / (1.0 + np.exp(z))

def run_channel(V, kappa, Vc):
    return sigmoid(kappa, V - Vc)

def aoi_positive_area(x):
    """AOI proxy: trapezoid integral of positive excursion above mean."""
    y = np.maximum(x - np.mean(x), 0.0)
    return float(np.trapezoid(y))

def ensure_dir(path):
    d = os.path.dirname(path)
    if d:
        os.makedirs(d, exist_ok=True)

# -------------------------
# Core AB vs BA schedule
# -------------------------
def run_AB_BA(
    V,
    Tsteps=120,
    charge_params=(2.4, 0.0),
    heat_params=(1.0, -0.15),
    p_thresh=0.7,
    alpha=0.0,
    asym_threshold=False,
    delta_thr=0.10,
    omega=0.8,
    drive_amp=0.25,
    seed=0,
):
    """
    AB vs BA:
      - AB: charge reads first, writes back, then heat reads
      - BA: heat reads first, writes back, then charge reads

    alpha=0 -> unordered control (no write-back)
    asym_threshold -> shifts VcC and VcH by +/- delta_thr
    """
    rng = np.random.default_rng(seed)  # present for future extension; not used by default
    V = np.asarray(V, dtype=float).copy()

    kC, VcC = charge_params
    kH, VcH = heat_params

    if asym_threshold:
        VcC = VcC + float(delta_thr)
        VcH = VcH - float(delta_thr)

    def drive(t):
        return (1.0 + float(drive_amp) * np.cos(float(omega) * t))

    def schedule(order):
        rC = np.zeros(Tsteps, dtype=float)
        rH = np.zeros(Tsteps, dtype=float)
        dR = np.zeros(Tsteps, dtype=float)  # reach difference

        for t in range(Tsteps):
            Vt = drive(t) * V

            if order == "AB":
                pC = run_channel(Vt, kC, VcC)
                Vt2 = Vt - float(alpha) * (pC - pC.mean()) if alpha != 0.0 else Vt
                pH = run_channel(Vt2, kH, VcH)
            else:  # "BA"
                pH = run_channel(Vt, kH, VcH)
                Vt2 = Vt - float(alpha) * (pH - pH.mean()) if alpha != 0.0 else Vt
                pC = run_channel(Vt2, kC, VcC)

            rC[t] = float(np.mean(pC > p_thresh))
            rH[t] = float(np.mean(pH > p_thresh))
            dR[t] = rC[t] - rH[t]

        return rC, rH, dR

    rC_AB, rH_AB, dR_AB = schedule("AB")
    rC_BA, rH_BA, dR_BA = schedule("BA")

    AOI_AB = aoi_positive_area(dR_AB)
    AOI_BA = aoi_positive_area(dR_BA)

    out = dict(
        AOI_gap_L1=float(abs(AOI_AB - AOI_BA)),
        AOI_gap_signed=float(AOI_AB - AOI_BA),
        AOI_gap_final=float(dR_AB[-1] - dR_BA[-1]),
        AOI_AB=float(AOI_AB),
        AOI_BA=float(AOI_BA),
        rC_AB=rC_AB, rH_AB=rH_AB, dR_AB=dR_AB,
        rC_BA=rC_BA, rH_BA=rH_BA, dR_BA=dR_BA,
        alpha=float(alpha),
        p_thresh=float(p_thresh),
        asym_threshold=bool(asym_threshold),
        delta_thr=float(delta_thr),
        omega=float(omega),
        drive_amp=float(drive_amp),
        seed=int(seed),
    )
    return out

# -------------------------
# Figure helpers
# -------------------------
def plot_quadrant_bars(stage_null, stage_ordered, labels, short_labels, out_png, p_thresh, alpha_null, alpha_ord):
    """
    Two-panel compact figure:
      Left: null controls (alpha=0) — show bars even if all zero
      Right: ordered (alpha>0)
    """
    ensure_dir(out_png)

    null_vals = np.array([stage_null[k]["AOI_gap_L1"] for k in labels], dtype=float)
    ord_vals  = np.array([stage_ordered[k]["AOI_gap_L1"] for k in labels], dtype=float)

    x = np.arange(len(labels), dtype=float)
    w = 0.72

    fig, axes = plt.subplots(1, 2, figsize=(7.0, 2.6), dpi=240, constrained_layout=True)

    # ---- Left (null)
    ax = axes[0]
    ax.bar(x, null_vals, width=w, facecolor="none", edgecolor="black", linewidth=1.6)
    ax.axhline(0.0, linewidth=1.0)
    ax.set_title(f"Controls (alpha={alpha_null:g})")
    ax.set_ylabel("AOI-gap")
    ax.set_xticks(x)
    ax.set_xticklabels(short_labels)

    # force visible scale even if exactly zero
    vmax = float(np.max(np.abs(null_vals)))
    span = max(0.02, 5.0 * vmax)
    ax.set_ylim(-span, span)

    for i, v in enumerate(null_vals):
        ax.text(i, v + 0.06*span, f"{v:.3f}", ha="center", va="bottom", fontsize=9)

    ax.text(0.02, 0.92, "Strict nulls shown\n(not omitted)", transform=ax.transAxes,
            ha="left", va="top", fontsize=9)

    # ---- Right (ordered)
    ax = axes[1]
    ax.bar(x, ord_vals, width=w, linewidth=0.0)
    ax.axhline(0.0, linewidth=1.0)
    ax.set_title(f"Ordered (alpha={alpha_ord:g})")
    ax.set_ylabel("AOI-gap")
    ax.set_xticks(x)
    ax.set_xticklabels(short_labels)

    top = max(1e-6, 1.18 * float(np.max(ord_vals)))
    ax.set_ylim(0.0, top)

    for i, v in enumerate(ord_vals):
        ax.text(i, v + 0.03*top, f"{v:.3f}", ha="center", va="bottom", fontsize=9)

    fig.suptitle(f"AOI-gap quadrant (p_thresh={p_thresh:g})", y=1.04, fontsize=12)
    plt.savefig(out_png, bbox_inches="tight")
    plt.close(fig)

def plot_traces_stacked(stage_ordered, labels, out_png, title):
    """
    Stacked traces: for each case, plot dR_AB (solid) and dR_BA (dashed).
    (This is the cleanest witness of non-commutativity for readers.)
    """
    ensure_dir(out_png)

    n = len(labels)
    fig, axes = plt.subplots(n, 1, figsize=(7.0, 6.6), dpi=240, sharex=True, constrained_layout=True)
    if n == 1:
        axes = [axes]

    for ax, lab in zip(axes, labels):
        dR_AB = np.asarray(stage_ordered[lab]["dR_AB"], dtype=float)
        dR_BA = np.asarray(stage_ordered[lab]["dR_BA"], dtype=float)

        ax.plot(dR_AB, label="AB", linewidth=1.4)
        ax.plot(dR_BA, "--", label="BA", linewidth=1.4)
        ax.axhline(0.0, linewidth=0.9)
        ax.set_ylabel(lab.replace("_", "\n"), rotation=0, labelpad=48, va="center", fontsize=8)

    axes[-1].set_xlabel("t")
    axes[0].legend(loc="upper right", fontsize=9, frameon=False)
    fig.suptitle(title, y=1.01, fontsize=12)

    plt.savefig(out_png, bbox_inches="tight")
    plt.close(fig)

# -------------------------
# FULL run: includes zero controls
# -------------------------
def run_paperX_full(
    outdir="outputs",
    Nx=23, Ny=23,
    Tsteps=120,
    p_thresh=0.7,
    alpha_null=0.0,
    alpha_ord=0.8,
    chi=0.6,
    delta_thr=0.10,
    seed=0,
):
    ensure_dir(os.path.join(outdir, "dummy.txt"))

    n = int(Nx) * int(Ny)
    x = np.linspace(0.0, 2.0*np.pi, n, endpoint=False)

    # Base minimal field
    V_base = np.cos(x)

    # Minimal "chirality" perturbation (handed harmonic)
    V_chiral = V_base + float(chi) * np.sin(2.0 * x)

    labels = [
        "SYM_graphene_graphene",
        "ASYM_threshold_only",
        "ASYM_chirality_only",
        "ASYM_chi_plus_threshold",
    ]
    short_labels = ["SYM", "THR", "CHI", "CHI+THR"]

    cases = [
        (V_base,   False),
        (V_base,   True),
        (V_chiral, False),
        (V_chiral, True),
    ]

    # -------- Stage NULL: strict controls
    stage_null = {}
    for lab, (V, asym_thr) in zip(labels, cases):
        stage_null[lab] = run_AB_BA(
            V=V,
            Tsteps=Tsteps,
            p_thresh=p_thresh,
            alpha=alpha_null,
            asym_threshold=asym_thr,
            delta_thr=delta_thr,
            seed=seed,
        )

    # -------- Stage ORDERED: non-commutative ordering on
    stage_ord = {}
    for lab, (V, asym_thr) in zip(labels, cases):
        stage_ord[lab] = run_AB_BA(
            V=V,
            Tsteps=Tsteps,
            p_thresh=p_thresh,
            alpha=alpha_ord,
            asym_threshold=asym_thr,
            delta_thr=delta_thr,
            seed=seed,
        )

    # -------- Save one NPZ summary
    out_npz = os.path.join(outdir, f"aoi_gap_quadrant_summary_alpha{alpha_null:g}_vs_alpha{alpha_ord:g}_pth{p_thresh:.2f}.npz")
    np.savez_compressed(
        out_npz,
        stage_null=stage_null,
        stage_ordered=stage_ord,
        labels=np.array(labels, dtype=object),
        short_labels=np.array(short_labels, dtype=object),
        Nx=int(Nx), Ny=int(Ny),
        Tsteps=int(Tsteps),
        p_thresh=float(p_thresh),
        alpha_null=float(alpha_null),
        alpha_ord=float(alpha_ord),
        chi=float(chi),
        delta_thr=float(delta_thr),
        seed=int(seed),
    )

    # -------- Figures
    fig2_png = os.path.join(outdir, "fig2_AOI_gap_quadrant_controls_vs_ordered.png")
    plot_quadrant_bars(
        stage_null=stage_null,
        stage_ordered=stage_ord,
        labels=labels,
        short_labels=short_labels,
        out_png=fig2_png,
        p_thresh=p_thresh,
        alpha_null=alpha_null,
        alpha_ord=alpha_ord,
    )

    fig3_png = os.path.join(outdir, "fig3_dReach_traces_ordered_stacked.png")
    plot_traces_stacked(
        stage_ordered=stage_ord,
        labels=labels,
        out_png=fig3_png,
        title=f"Ordered-access traces (alpha={alpha_ord:g}, p_thresh={p_thresh:g})",
    )

    # -------- Console summary (so you can paste into paper SI)
    print("=== AOI-gap quadrant (strict controls + ordered) ===")
    for lab in labels:
        z = stage_null[lab]["AOI_gap_L1"]
        o = stage_ord[lab]["AOI_gap_L1"]
        print(f"{lab:28s}  control={z: .6f}   ordered={o: .6f}")

    print("\nSaved:", out_npz)
    print("Saved:", fig2_png)
    print("Saved:", fig3_png)

    return out_npz, fig2_png, fig3_png

# ============================================================
# RUN (default witness parameters)
# ============================================================
run_paperX_full()

=== AOI-gap quadrant (strict controls + ordered) ===
SYM_graphene_graphene         control= 0.000000   ordered= 1.503261
ASYM_threshold_only           control= 0.000000   ordered= 3.676244
ASYM_chirality_only           control= 0.000000   ordered= 1.780120
ASYM_chi_plus_threshold       control= 0.000000   ordered= 1.292155

Saved: outputs/aoi_gap_quadrant_summary_alpha0_vs_alpha0.8_pth0.70.npz
Saved: outputs/fig2_AOI_gap_quadrant_controls_vs_ordered.png
Saved: outputs/fig3_dReach_traces_ordered_stacked.png


('outputs/aoi_gap_quadrant_summary_alpha0_vs_alpha0.8_pth0.70.npz',
 'outputs/fig2_AOI_gap_quadrant_controls_vs_ordered.png',
 'outputs/fig3_dReach_traces_ordered_stacked.png')